# 🛰️ Methane Shadow Hunter - End-to-End Logic Demo
This notebook demonstrates the core logic pipeline for detecting, attributing, and quantifying methane super-emitters, mapping them to oil & gas infrastructure, and generating LangChain-powered compliance audits.

In [ ]:
import sys
from pathlib import Path

# Ensure the src/ folder is in path
sys.path.insert(0, str(Path().resolve()))

from src.config import config
from src.data.sentinel5p import Sentinel5PClient
from src.data.carbonmapper import CarbonMapperClient
from src.data.infrastructure import InfrastructureDB
from src.fusion.hotspot_detector import HotspotDetector
from src.fusion.tasking_simulator import TaskingSimulator
from src.fusion.spatial_join import SpatialJoiner
from src.plume.inversion import PlumeInverter
from src.plume.wind import WindField
from src.agent.reporting_agent import ComplianceAuditAgent

### 1. Data Ingestion: Coarse Sentinel-5P Hotspots
First, we load the `India_Methane_Hotspots.csv` dataset, parsing the geometries and calculating baseline anomaly scores.

In [ ]:
s5p_client = Sentinel5PClient()
hotspots_df = s5p_client.load_hotspots_csv()
display(hotspots_df.head())

print("\nDataset Summary Stats:")
print(s5p_client.get_summary_stats())

### 2. Multi-Scale Data Fusion: Hotspot Detection & Tasking
Filter standard methane anomalies from genuine "Super-Emiters" using a Sigma threshold.
Then trigger a simulated high-res tasking request (e.g., GHGSat, Planet).

In [ ]:
detector = HotspotDetector(threshold_sigma=2.0)
detected = detector.detect(hotspots_df)
tasking_candidates = detector.get_tasking_candidates(detected)

print("Detection Summary:")
print(detector.summary(detected))

# Simulate tasking high-res satellites for top 5 candidates
tasking = TaskingSimulator()
requests = tasking.create_tasking_requests(tasking_candidates, max_requests=5)
print(f"\nCreated {len(requests)} tasking requests. Example:", requests[0])

### 3. High-Res Plume Attribution (Spatial Join)
Using CarbonMapper STAC AI to find high-resolution plumes and joined with local oil & gas infrastructure.

In [ ]:
cm_client = CarbonMapperClient()
infra_db = InfrastructureDB()
joiner = SpatialJoiner(radius_km=5.0)

hotspot_coords = [(h.latitude, h.longitude) for h in tasking_candidates[:5]]
plumes = cm_client.generate_synthetic_plumes(hotspot_coords) # Uses synthetic for offline demo

facilities = infra_db.load_facilities()
attributed = joiner.join(plumes, facilities)

print(f"Attributed {len(attributed)} plumes to infrastructure.")
if attributed:
    print("\nTop Attributed Emitter:")
    print(f"Facility: {attributed[0].facility_name} ({attributed[0].operator})")
    print(f"Distance: {attributed[0].pinpoint_accuracy_m}m")
    print(f"Emission Rate: {attributed[0].emission_rate_kg_hr} kg/hr")

### 4. Plume Inversion Modeling (PyTorch)
Using a differentiable Gaussian Plume Model to reverse-engineer the actual emission rate `Q` (kg/hr).

In [ ]:
wind = WindField()
inverter = PlumeInverter()

if attributed:
    attr = attributed[0]
    wind_data = wind.get_wind(attr.plume_lat, attr.plume_lon)
    
    print("Running Plume Inversion Optimization in PyTorch...")
    true_Q_kg_s = attr.emission_rate_kg_hr / 3600
    synth = inverter.create_synthetic_observation(
        true_Q_kg_s=true_Q_kg_s, 
        wind_speed=wind_data.speed_ms,
        stability_class=wind_data.stability_class
    )
    
    result = inverter.invert(
        observed_concentrations=synth["observed_concentrations"],
        receptor_x=synth["receptor_x"],
        receptor_y=synth["receptor_y"],
        receptor_z=synth["receptor_z"],
        wind_speed=wind_data.speed_ms,
        initial_Q=0.01,
        true_Q_kg_hr=synth["true_Q_kg_hr"]
    )
    
    print(f"Target Emission:  {synth['true_Q_kg_hr']:.2f} kg/hr")
    print(f"Estimated (Inv):  {result.estimated_Q_kg_hr:.2f} kg/hr")
    print(f"Error Margin:     {result.error_pct}%\n")

### 5. Autonomous Reporting Agent (LangChain + Featherless AI)
The data is passed to a Featherless AI LLM agent (OpenAI-compatible) to create a structured compliance audit.

In [ ]:
agent = ComplianceAuditAgent(
    model=config.featherless_model,
    api_key=config.featherless_api_key,
    base_url=config.featherless_base_url,
)

if attributed:
    report = agent.generate_report(attributed[0], plumes[0])
    print("\n================ AUDIT REPORT ================")
    print(report.report_markdown[:700] + "\n\n[...Report Trimmed For Display...]")